# Monte Carlo estimation of Sensitivities in Finance
## MATH-414 Stochastic Simulation project
Authors : Charles Gendreau - Eliott Van Dieren

Professor : Prof. Fabio Nobile

Teaching Assistant : Sundar Ganesh

In [3]:
# Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as st

In [8]:
# Utility functions

def simul_S_T(T:float,S_0:float,r:float,sig:float,n:int) -> np.array:
    """
    Returns a numpy array of size n with prices of the underlying at maturity
    
    Args :
        - T : Time of maturity (% of years) (float)
        - S_0 : Underlying price at time 0 (float)
        - r : free-risk interest rate (float)
        - sig : Volatility of the underlying (float)
        - n : number of prices
    Returns :
        - numpy array of S_T
    """
    W_T : np.array = st.norm.rvs(loc=0,scale=T**2,size=n)
    return S_0*np.exp((r-0.5*sig**2)*T+sig*W_T)

def CMC_estimator(func,X:np.array) -> list:
    """
    Crude Monte-Carlo estimator of E[func(X)]
    where N = length of X
    
    args:
        - func : function from R^N to R^N
        - X : numpy array
    returns : Crude Monte-Carlo estimator
    
    """
    return [np.mean(func(X)),np.std(func(X))]

#def finite_difference(fun,delta:float) 

## 3. Application to option pricing

### 3.1 European Call Option

In [54]:
# Parameters
T = 1
S_0 = 100
K = 120
r = 0.05
sig = 0#0.25
n_elem = np.arange(10**3,10**5+1000,1000)
#print(n_elem)

# Payoff function
#f = lambda x: np.exp(-r*T)*np.maximum(x-K,np.zeros(len(x)))
f = lambda x : x
# Vraies functions : ∂f(ST )/∂S0, ∂f(ST)/∂σ, ∂2f(ST )/∂S02

dS_0 = 1e-3

# Generation sampling
#plot_S_T_mu = np.zeros(len(n_elem))
#plot_S_T_std = np.zeros(len(n_elem))
dI_d_S_T = np.zeros(len(n_elem))
test = np.zeros(len(n_elem))
for idx,n in enumerate(n_elem):
    S_T_r = simul_S_T(T,S_0+dS_0,r,sig,n)
    S_T_l = simul_S_T(T,S_0-dS_0,r,sig,n)
    test[idx] = simul_S_T(T,S_0,r,sig,n)[0]
    [payoff_mu_r,payoff_std_r] = CMC_estimator(f,S_T_r)
    [payoff_mu_l,payoff_std_l] = CMC_estimator(f,S_T_l)
    dI_d_S_T[idx] = (payoff_mu_r-payoff_mu_l)/(2*dS_0)

In [56]:
#plt.figure(figsize=(10,5))
#plt.plot(n_elem,dI_d_S_T)
#plt.plot(n_elem,np.exp(r*T)*np.ones(len(n_elem)))
#plt.show()